# Build an Interactive Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
TASK 3: Add a Range Slider to Select Payload
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

Which site has the largest successful launches?
Which site has the highest launch success rate?
Which payload range(s) has the highest launch success rate?
Which payload range(s) has the lowest launch success rate?
Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

Run the following wget command line in the terminal to download dataset as spacex_launch_dash.csv

In [9]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url)

# Save the CSV content to a file
with open("spacex_launch_dash.csv", "wb") as file:
    file.write(response.content)

In [11]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
response = requests.get(url)

# Save the CSV content to a file
with open("spacex_launch_dash_app.csv", "wb") as file:
    file.write(response.content)

# TASK 1: Add a Launch Site Drop-down Input Component

We have four different launch sites and we would like to first see which one has the largest success count. Then,
we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

Find and complete a commented dcc.Dropdown(id='site-dropdown',...) input with following attributes:
id attribute with value site-dropdown
options attribute is a list of dict-like option objects (with label and value attributes). You can set
the label and value all to be the launch site names in the spacex_df
and you need to include the default All option. e.g.,

In [20]:
options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'site1', 'value': 'site1'}, ...]

value attribute with default dropdown value to be ALL meaning all sites are selected
placeholder attribute to show a text description about this input area,
such as Select a Launch Site here
searchable attribute to be True so we can enter keywords to search launch sites
Here is an example of dcc.Dropdown:

In [7]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd

# Load data
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a list of launch sites for the dropdown options
launch_sites = spacex_df['Launch Site'].unique()
launch_site_options = [{'label': 'All Sites', 'value': 'ALL'}]
launch_site_options += [{'label': site, 'value': site} for site in launch_sites]

# Define the layout of the app
app.layout = html.Div([
    # Dropdown for selecting a launch site
    html.Label("Select Launch Site:"),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_site_options,
        value='ALL',  # Default value
        placeholder="Select a launch site",
        searchable=True
    ),
    
    # Output area
    html.Div(id='output-container')
])

# Define callback to update output based on dropdown selection
@app.callback(
    Output('output-container', 'children'),
    Input('site-dropdown', 'value')
)
def update_output(selected_site):
    # Filter data based on the selected launch site
    if selected_site == 'ALL':
        filtered_df = spacex_df
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
    
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

The general idea of this callback function is to get the selected launch site from site-dropdown and render
a pie chart visualizing launch success counts.

Dash callback function is a type of Python function which will be automatically called by
Dash whenever receiving an input component updates, such as a click or dropdown selecting event.

Let’s add a callback function in spacex_dash_app.py including the following application logic:

Input is set to be the site-dropdown dropdown, i.e., Input(component_id='site-dropdown', component_property='value')
Output to be the graph with id success-pie-chart, i.e., Output(component_id='success-pie-chart', component_property='figure')
A If-Else statement to check if ALL sites were selected or just a specific launch site was selected
If ALL sites are selected, we will use all rows in the dataframe spacex_df to render and return a pie chart graph to show the total success launches (i.e., the total count of class column)
If a specific launch site is selected, you need to filter the dataframe spacex_df first in order
to include the only data for the selected site.
Then, render and return a pie chart graph to show the success (class=1) count and failed (class=0) count for the selected site.
Here is an example of a callback function:

In [5]:
# Import necessary components
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load data
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a list of launch sites for the dropdown options
launch_sites = spacex_df['Launch Site'].unique()
launch_site_options = [{'label': 'All Sites', 'value': 'ALL'}]
launch_site_options += [{'label': site, 'value': site} for site in launch_sites]

# Define the layout of the app
app.layout = html.Div([
    # Dropdown for selecting a launch site
    html.Label("Select Launch Site:"),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_site_options,
        value='ALL',  # Default value
        placeholder="Select a launch site",
        searchable=True
    ),
    
    # Output area
    html.Div(id='output-container')
])

# Define callback to update output based on dropdown selection
@app.callback(
    Output('output-container', 'children'),
    Input('site-dropdown', 'value')
)
def update_output(selected_site):
    # Filter data based on the selected launch site
    if selected_site == 'ALL':
        filtered_df = spacex_df
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]

# Function decorator to specify function input and output
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def get_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(
            spacex_df,
            names='class', 
            title="Total Success Launches for All Sites",
            hole=0.3
        )
        fig.update_traces(textinfo='percent+label')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        outcome_counts = filtered_df['class'].value_counts().reset_index()
        outcome_counts.columns = ['Outcome', 'Count']
        fig = px.pie(
            outcome_counts,
            names='Outcome',
            values='Count',
            title=f"Launch Success Rate for Site {selected_site}",
            hole=0.3
        )
        fig.update_traces(textinfo='percent+label')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(data, values='class', 
        names='pie chart names', 
        title='title')
        return fig
    else:
        # return the outcomes piechart for a selected site

In [51]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load the SpaceX launch data
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a dropdown list of launch sites
launch_sites = spacex_df['Launch Site'].unique()
launch_site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Define the layout of the app
app.layout = html.Div([
    # Dropdown for selecting a launch site
    html.Label("Select Launch Site:"),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_site_options,
        value='ALL',  # Default value
        placeholder="Select a launch site",
        searchable=True
    ),
    
    # Pie chart to visualize success counts
    dcc.Graph(id='success-pie-chart')
])

# Define the callback function
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(selected_site):
    # If "ALL" sites are selected, use the entire DataFrame
    if selected_site == 'ALL':
        # Count success and failure for all launches
        success_counts = spacex_df['class'].value_counts()
        
        # Create the pie chart for all sites
        fig = px.pie(
            names=['Failed', 'Successful'],
            values=success_counts,
            title='Total Launch Successes for All Sites'
        )
    else:
        # Filter the DataFrame for the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        
        # Count success and failure for the selected site
        success_counts = filtered_df['class'].value_counts()
        
        # Create the pie chart for the selected site
        fig = px.pie(
            names=['Failed', 'Successful'],
            values=success_counts,
            title=f'Launch Successes for {selected_site}'
        )
        
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [21]:
import plotly.express as px
from dash import Dash  # Using Dash instead of JupyterDash
import dash_html_components as html

# Instantiate Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div(children=[
    html.H1("SpaceX Launch Records Dashboard"),
    # Your additional components here
])

# Run the app on a different port to avoid conflicts
if __name__ == "__main__":
    app.run_server(port=8051, debug=True)


# Read the airline data into pandas dataframe
# spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
# app = dash.Dash(__name__) # commented out as we are using JupyterDash instead of dash.Dash
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             # style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', host='localhost')
    app.run_server(port=8051)  # Change 8051 to any other available port if needed

C:\Users\utente\anaconda3\Lib\site-packages\dash\dash.py:585: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



TypeError: 'NoneType' object cannot be interpreted as an integer

In [25]:
# Import Dash instead of JupyterDash
from dash import Dash
import dash_html_components as html
from dash import dcc

# Initialize the Dash app
app = Dash(__name__)

# Define the layout for the app
app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard'),
    dcc.Dropdown(id='site-dropdown', 
                 options=[{'label': 'All Sites', 'value': 'ALL'},
                          {'label': 'Site 1', 'value': 'site1'}],
                 value='ALL', 
                 placeholder="Select a Launch Site"),
    dcc.Graph(id='success-pie-chart'),
    # Additional components...
])

# Define the callback functions as before
# @app.callback(...) 
# def update_pie_chart(...): 

# Run the Dash app without 'inline' mode
if __name__ == '__main__':
    app.run_server(port=8051, debug=True)

In [27]:
!pip install dash
!pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly

Exception in thread Thread-195 (<lambda>):
Traceback (most recent call last):
  File "C:\Users\utente\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\utente\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\utente\anaconda3\Lib\site-packages\dash\dash.py", line 1857, in <lambda>
  File "C:\Users\utente\anaconda3\Lib\site-packages\dash\_watch.py", line 35, in watch
  File "C:\Users\utente\anaconda3\Lib\site-packages\dash\_watch.py", line 20, in walk
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\utente\\anaconda3\\Lib\\site-packages\\dash\\dash_table\\async-export.js.map'


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\utente\\anaconda3\\Lib\\site-packages\\dash\\deps/polyfill@7.12.1.min.js'

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\utente\\anaconda3\\Lib\\site-packages\\dash\\deps/polyfill@7.12.1.min.js'

     ---------------------------------------- 0.0/75.3 kB ? eta -:--:--
     -------------------------------------- - 71.7/75.3 kB 4.1 MB/s eta 0:00:01
     ---------------------------------------- 75.3/75.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------

In [ ]:
# notebook hangs if I run the following command 
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# needs to be run again in a separate cell due to a jupyterdash bug
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

In [ ]:
# skeleton Dash app to be completed in this lab, downloaded from:
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
# redundant lines commented out

# Import required libraries
# import pandas as pd
# import dash
# import dash_html_components as html
# import dash_core_components as dcc
# from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
# spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
# app = dash.Dash(__name__) # commented out as we are using JupyterDash instead of dash.Dash
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             # style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', host='localhost')

# TASK 3: Add a Range Slider to Select Payload

Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we
want to be able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented dcc.RangeSlider(id='payload-slider',...) input with the following attribute:

id to be payload-slider
min indicating the slider starting point, we set its value to be 0 (Kg)
max indicating the slider ending point to, we set its value to be 10000 (Kg)
step indicating the slider interval on the slider, we set its value to be 1000 (Kg)
value indicating the current selected range, we could set it to be min_payload and max_payload
Here is an example of RangeSlider:

In [63]:
import dash
from dash import dcc, html

# Load the SpaceX launch data
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a dropdown list of launch sites
launch_sites = spacex_df['Launch Site'].unique()
launch_site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Define the layout of the app
app.layout = html.Div([
    # Dropdown for selecting a launch site
    html.Label("Select Launch Site:"),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_site_options,
        value='ALL',  # Default value
        placeholder="Select a launch site",
        searchable=True
    ),
    
    # Pie chart to visualize success counts
    dcc.Graph(id='success-pie-chart')
])

# Define the callback function
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(selected_site):
    # If "ALL" sites are selected, use the entire DataFrame
    if selected_site == 'ALL':
        # Count success and failure for all launches
        success_counts = spacex_df['class'].value_counts()
        
        # Create the pie chart for all sites
        fig = px.pie(
            names=['Failed', 'Successful'],
            values=success_counts,
            title='Total Launch Successes for All Sites'
        )
    else:
        # Filter the DataFrame for the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        
        # Count success and failure for the selected site
        success_counts = filtered_df['class'].value_counts()
        
        # Create the pie chart for the selected site
        fig = px.pie(
            names=['Failed', 'Successful'],
            values=success_counts,
            title=f'Launch Successes for {selected_site}'
        )
        
    return fig

# Define the layout of the app
app.layout = html.Div([
    # Add the payload RangeSlider
    html.Label("Select Payload Range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',  # Set the id for the RangeSlider
        min=0,  # Starting point of the slider
        max=10000,  # Ending point of the slider
        step=1000,  # Interval for the slider
        marks={i: str(i) for i in range(0, 10001, 1000)},  # Marks at each 1000 Kg increment
        value=[0, 10000]  # Default range selected (0 to 10000 Kg)
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)